"¿Cómo influyen los indicadores económicos y demográficos en la tasa de crecimiento de la población a nivel global?"

In [32]:
# Importar librerías necesarias
from pgmpy.models import BayesianNetwork
from pgmpy.factors.discrete import TabularCPD
from pgmpy.inference import VariableElimination

In [33]:
# Definir la estructura de la red bayesiana
LV_URB = "Nivel urbanización"
LV_EDU = "Nivel educativo"
RT_PARO = "Tasa de paro"
PNB = "Ingreso nacional bruto"
PIB = "Producto Interior Bruto"
GASTO_EDUCATIVO = "Gasto educativo"
ACCESO_SALUD = "Acceso a salud"
RT_FERTILIDAD = "Tasa de fertilidad"
RT_CRECIMIENTO = "Tasa de crecimiento de la población"
RT_MORT_INFANTIL = "Tasa de mortalidad infantil"

# Se le pasa por parámetro una lista de tuplas (origen --> destino)
model = BayesianNetwork()
model.add_nodes_from([LV_URB, LV_EDU, RT_PARO, PNB, PIB, GASTO_EDUCATIVO, 
                      ACCESO_SALUD, RT_FERTILIDAD, RT_CRECIMIENTO, RT_MORT_INFANTIL])

edges_RT_PARO = [(LV_EDU, RT_PARO), (LV_URB, RT_PARO)]
edges_PIB = [(RT_PARO, PIB), (PNB, PIB)]
edges_RT_FERTILIDAD = [(GASTO_EDUCATIVO, RT_FERTILIDAD), (ACCESO_SALUD, RT_FERTILIDAD)]
edges_RT_CRECIMIENTO = [(RT_FERTILIDAD, RT_CRECIMIENTO), (RT_MORT_INFANTIL, RT_CRECIMIENTO)]

model.add_edges_from(edges_RT_PARO)
model.add_edges_from(edges_PIB)
model.add_edge(PIB, GASTO_EDUCATIVO)
model.add_edges_from(edges_RT_FERTILIDAD)
model.add_edges_from(edges_RT_CRECIMIENTO)


In [34]:
# Suponiendo que ya tienes los datos para cada nodo, puedes definir los CPD
# Aquí, para simplificación, se usarán datos ficticios
#------------------------------------------------------------------
cpds = dict()
#------------------------------------------------------------------
cpd_nivel_educativo = TabularCPD(
    variable=LV_EDU, variable_card=2, 
    values=[[0.7], 
            [0.3]],
    state_names={LV_EDU: ['ALTO', 'BAJO']}
    )
cpds[LV_EDU] = cpd_nivel_educativo
#------------------------------------------------------------------
cpd_nivel_urbanizacion = TabularCPD(
    variable=LV_URB, variable_card=2,
    values=[[0.75], 
            [0.25]],
    state_names={LV_URB: ['ALTO', 'BAJO']}
    )
cpds[LV_URB] = cpd_nivel_urbanizacion
#------------------------------------------------------------------
cpd_tasa_paro_cond = TabularCPD(
    variable=RT_PARO, variable_card=2,
    values=[[0.1, 0.15, 0.3, 0.95],
            [0.9, 0.85, 0.7, 0.05]],
    evidence=[LV_EDU, LV_URB],
    evidence_card=[2, 2],
    state_names={RT_PARO: ['ALTO', 'BAJO'],
                 LV_EDU: ['ALTO', 'BAJO'],
                 LV_URB: ['ALTO', 'BAJO']}
    )
cpds[RT_PARO] = cpd_tasa_paro_cond
#------------------------------------------------------------------
cpd_pnb_per_capita = TabularCPD(
    variable=PNB, variable_card=2, 
    values=[[0.7], 
            [0.3]],
    state_names={PNB: ['ALTO', 'BAJO']}
    )
cpds[PNB] = cpd_pnb_per_capita
#------------------------------------------------------------------
cpd_pib_per_capita_cond = TabularCPD(
    variable=PIB, variable_card=2,
    values=[[0.8, 0.5, 0.3, 0.1],
            [0.2, 0.5, 0.7, 0.9]],
    evidence=[RT_PARO, PNB],
    evidence_card=[2, 2],
    state_names={RT_PARO: ['ALTO', 'BAJO'],
                 PNB: ['ALTO', 'BAJO'],
                 PIB: ['ALTO', 'BAJO']}
    )
cpds[PIB] = cpd_pib_per_capita_cond
#------------------------------------------------------------------
cpd_gasto_educacion_cond = TabularCPD(
    variable=GASTO_EDUCATIVO, variable_card=2, 
    values=[[0.7, 0.3], 
            [0.3, 0.7]], 
    evidence=[PIB], 
    evidence_card=[2],
    state_names={GASTO_EDUCATIVO: ['ALTO', 'BAJO'],
                 PIB: ['ALTO', 'BAJO']}
    )
cpds[GASTO_EDUCATIVO] = cpd_gasto_educacion_cond
#------------------------------------------------------------------
cpd_acceso_salud = TabularCPD(
    variable=ACCESO_SALUD, variable_card=2, 
    values=[[0.6], 
            [0.4]],
    state_names={ACCESO_SALUD: ['ALTO', 'BAJO']})
cpds[ACCESO_SALUD] = cpd_acceso_salud
#------------------------------------------------------------------
cpd_tasa_fertilidad_cond = TabularCPD(
    variable=RT_FERTILIDAD, variable_card=2,
    values=[[0.8, 0.15, 0.7, 0.5],
            [0.2, 0.85, 0.3, 0.5]],
    evidence=[GASTO_EDUCATIVO, ACCESO_SALUD],
    evidence_card=[2, 2],
    state_names={GASTO_EDUCATIVO: ['ALTO', 'BAJO'],
                 ACCESO_SALUD: ['SI', 'NO'],
                 RT_FERTILIDAD: ['ALTO', 'BAJO']}
    )
cpds[RT_FERTILIDAD] = cpd_tasa_fertilidad_cond
#------------------------------------------------------------------
cpd_tasa_mortalidad_infantil = TabularCPD(
    variable=RT_MORT_INFANTIL, variable_card=2, 
    values=[[0.8], 
            [0.2]],
    state_names={RT_MORT_INFANTIL: ['ALTO', 'BAJO']}
    )
cpds[RT_MORT_INFANTIL] = cpd_tasa_mortalidad_infantil
#------------------------------------------------------------------
cpd_tasa_crecimiento_poblacion_cond = TabularCPD(
    variable=RT_CRECIMIENTO, variable_card=2,
    values=[[0.7, 0.9, 0.25, 0.17],
            [0.3, 0.1, 0.75, 0.83]],
    evidence=[RT_FERTILIDAD, RT_MORT_INFANTIL],
    evidence_card=[2, 2],
    state_names={RT_FERTILIDAD: ['ALTO', 'BAJO'],
                 RT_MORT_INFANTIL: ['ALTO', 'BAJO'],
                 RT_CRECIMIENTO: ['ALTO', 'BAJO']}
)
cpds[RT_CRECIMIENTO] = cpd_tasa_crecimiento_poblacion_cond
#------------------------------------------------------------------

In [35]:
# Pasasmos todos los datos a csv para mantenerlos guardados de una forma que sea más faciles de visualizar
for key, value in cpds.items():
    value.to_csv(filename=f'./csv/{key}.csv')

In [36]:
# Añadir todos los CPD al modelo
for key, value in cpds.items():
    print(key)
    model.add_cpds(value)

# Verificar que el modelo sea válido
assert model.check_model()

# Realizar inferencias
infer = VariableElimination(model)
result = infer.map_query(variables=[RT_CRECIMIENTO])
print(result)

Nivel educativo
Nivel urbanización
Tasa de paro
Ingreso nacional bruto
Producto Interior Bruto
Gasto educativo
Acceso a salud
Tasa de fertilidad
Tasa de mortalidad infantil
Tasa de crecimiento de la población


ValueError: The state names of Acceso a salud doesn't match in it's child nodes.